In [21]:
%load_ext autoreload
%autoreload 2
from infra.amazondatasetloader import AmazonReviewsDatasetLoader
from infra.svdpredictor import SVDRatingPredictor
from core.util import task

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dataset = AmazonReviewsDatasetLoader().load_dataset()
train, test = dataset.split([0.8, 0.2])

 ⟍  Amazon review data load
   ⟍  Raw data loading
   ⟋  Raw data loading [+6856ms]
   ⟍  Data conversion
    |  Loaded 50000 ratings (0.10%) [+12984ms]
    |  Loaded 100000 ratings (0.20%) [+3500ms]
    |  Loaded 150000 ratings (0.30%) [+3425ms]
    |  Loaded 200000 ratings (0.40%) [+2528ms]
    |  Loaded 250000 ratings (0.50%) [+1923ms]
    |  Loaded 300000 ratings (0.61%) [+1667ms]
    |  Loaded 350000 ratings (0.71%) [+1523ms]
    |  Loaded 400000 ratings (0.81%) [+1686ms]
    |  Loaded 450000 ratings (0.91%) [+1523ms]
    |  Loaded 500000 ratings (1.01%) [+1469ms]
    |  Loaded 550000 ratings (1.11%) [+1770ms]
    |  Loaded 600000 ratings (1.21%) [+1451ms]
    |  Loaded 650000 ratings (1.31%) [+1569ms]
    |  Loaded 700000 ratings (1.41%) [+1440ms]
    |  Loaded 750000 ratings (1.51%) [+1466ms]
    |  Loaded 800000 ratings (1.62%) [+1445ms]
    |  Loaded 850000 ratings (1.72%) [+1539ms]
    |  Loaded 900000 ratings (1.82%) [+1508ms]
    |  Loaded 950000 ratings (1.92%) [+1415ms]
 

In [25]:
history = []
model_path = "model.npz"
predictor = SVDRatingPredictor(latent_dim=2)
try:
    predictor.load(model_path)
except:
    pass

 ⟍  Model initialization
  |  latent_dim : 2 [+0ms]
  |  gamma      : 0.05 [+0ms]
 ⟋  Model initialization [+0ms]


In [23]:
with task("Training"):
    for i in range(1000):
        with task("Epoch %d" % (i + 1)):
            try:
                predictor.train(train)
                train_mse = predictor.evaluate(train)
                valid_mse = predictor.evaluate(test)
                history.append((train_mse, valid_mse))
                task.log("Train MSE      : %.4f" % train_mse)
                task.log("Validation MSE : %.4f" % valid_mse)
                predictor.save(model_path)
            except KeyboardInterrupt:
                task.log("Training interrupted")
                break

 ⟍  Training
   ⟍  Epoch 1


/Users/unknownpgr/Desktop/rating/src/infra/svdpredictor.py:46: RuntimeWarning: overflow encountered in multiply
  U[u] += gamma * (error * V[i] - lambda_ * U[u])
/Users/unknownpgr/Desktop/rating/src/infra/svdpredictor.py:47: RuntimeWarning: overflow encountered in multiply
  V[i] += gamma * (error * U[u] - lambda_ * V[i])
/Users/unknownpgr/Desktop/rating/src/infra/svdpredictor.py:47: RuntimeWarning: invalid value encountered in subtract
  V[i] += gamma * (error * U[u] - lambda_ * V[i])
/Users/unknownpgr/Desktop/rating/src/infra/svdpredictor.py:46: RuntimeWarning: invalid value encountered in add
  U[u] += gamma * (error * V[i] - lambda_ * U[u])


    |  Training interrupted [+15347ms]
   ⟋  Epoch 1 [+15348ms]
 ⟋  Training [+15348ms]


In [ ]:
import matplotlib.pyplot as plt

train_mse, valid_mse = zip(*history)
plt.plot(valid_mse, label="Validation MSE")
plt.legend()
plt.show()